In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_columns = 500

# Feature Egineering 
### - Create time from last fight
### - ordinal encode Max_round
### - HomeTown will be logitude latitude 
### - Vectorize Name might help 

In [3]:
df = pd.read_csv('clean_fights')
df.head()

,BPrev,BStreak,B_Age,B_Height,B_HomeTown,B_ID,B_Location,B_Name,B_Weight,Date,Event_ID,Fight_ID,Last_round,Max_round,RPrev,R_Age,R_Height,R_HomeTown,R_ID,R_Location,R_Name,R_Weight,winby,winner,avg_weight_lbs,weight_class
0,0,0,38.0,193.0,Hounslow England,808,Amsterdam The Netherlands,Alistair Overeem,120.0,02/03/2014,646,4580,3,3,0,39.0,190.0,"Las Vegas, Nevada USA",377,"Las Vegas, Nevada USA",Frank Mir,119.0,DEC,blue,262.9,Cruiserweight
1,0,0,36.0,172.0,"Chicago, Illinois United States",1054,"Chicago, Illinois United States",Ricardo Lamas,65.0,02/03/2014,646,4589,5,5,0,32.0,170.0,Manaus Brazil,1052,Rio de Janeiro Brazil,Jose Aldo,65.0,DEC,red,143.0,Bantamweight
2,0,0,39.0,167.0,"Isla Vista , California USA",959,"Sacramento, California USA",Urijah Faber,61.0,02/03/2014,646,4590,1,5,0,31.0,167.0,Natal Brazil,1527,Rio de Janeiro Brazil,Renan Barao,61.0,KO/TKO,red,134.2,Flyweight
3,0,0,33.0,167.0,"San Diego, CA USA",1056,"San Diego, CA USA",Danny Martinez,56.0,02/03/2014,646,4605,3,3,0,37.0,160.0,"San Jose, California USA",1253,"Tucson, Arizona USA",Chris Cariaso,56.0,DEC,red,123.2,Strawwweight
4,0,0,36.0,185.0,Southampton England,2005,Southampton England,Tom Watson,84.0,02/03/2014,646,4631,3,3,0,37.0,182.0,"Englewood, NJ USA",464,"Brick, NJ USA",Nick Catone,84.0,DEC,red,184.8,Super Welterweight


## Vectorize names with word to vec

In [16]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings 
  
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec

In [24]:
names = df.head().B_Name.values

In [25]:
names

array(['Alistair Overeem', 'Ricardo Lamas', 'Urijah Faber',
       'Danny Martinez', 'Tom Watson'], dtype=object)

In [29]:
tokens = [word_tokenize(x) for x in names]
tokens

[['Alistair', 'Overeem'],
 ['Ricardo', 'Lamas'],
 ['Urijah', 'Faber'],
 ['Danny', 'Martinez'],
 ['Tom', 'Watson']]

In [28]:
[Word2Vec(x) for x in tokens]

RuntimeError: you must first build vocabulary before training the model

## Use Geopy to add longitude and latitude based off HomeTown

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='pwalis', timeout=3)
location = geolocator.geocode("Brazil", addressdetails=True)


In [ ]:
one = location


In [ ]:
import time
start = time.process_time()
[geolocator.geocode(x) for x in df.R_HomeTown.head(30)]
print(time.process_time() - start)

In [ ]:
R_geo = []
i = 0
for x in df.R_HomeTown:
    i+=1
    print(i)
    geo_info = geolocator.geocode(x)
    R_geo.append(geo_info)
# honestly I just want a green square

A lot of places aren't getting recognized as a location I should look into how the api expects the input
Research how to handle different errors 
if statements based on the error, If it can't get the location info add an entry into the dict that has the hometown string

In [27]:
from geopy.exc import GeocoderTimedOut
i = 0
R_geo = dict()
for x in df.R_HomeTown:
    i+=1 
    try:
        location = geolocator.geocode(x).raw
        R_geo.update({i:location})
    except Exception as e:
        print("Error: geocode failed on input %s with message %s"%(x, e))
        

Error: geocode failed on input Espiritos Santos Brazil with message 'NoneType' object has no attribute 'raw'
Error: geocode failed on input Celle Saint Cloud, Paris France with message 'NoneType' object has no attribute 'raw'
Error: geocode failed on input Nagoya, Aichi Pref. Japan with message 'NoneType' object has no attribute 'raw'
Error: geocode failed on input Pine Ridge, Sout Dakota USA with message 'NoneType' object has no attribute 'raw'
Error: geocode failed on input Nagoya, Aichi Pref. Japan with message 'NoneType' object has no attribute 'raw'
Error: geocode failed on input Azle, Texas Untied States with message 'NoneType' object has no attribute 'raw'
Error: geocode failed on input Espiritos Santos Brazil with message 'NoneType' object has no attribute 'raw'
Error: geocode failed on input Montego Bay, Jamaica West Indies with message 'NoneType' object has no attribute 'raw'
Error: geocode failed on input Pine Ridge, Sout Dakota USA with message 'NoneType' object has no attr

KeyboardInterrupt: 

In [29]:
len(R_geo)

1161

In [38]:
i=0
while i<1162:
    i+=1
    if len(R_geo[i])<10:
        print(R_geo[i])
        

KeyError: 14

In [42]:
R_geo.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 